<h2>Importing libraries</h2>

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<h3>Getting the contents of the page</h3>

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url).text

<h3>Importing beautifulsoup library</h3>

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page,'lxml')

<h3>Extracting table using Beautifulsoup</h3>

In [13]:
my_table = soup.find('table') # Finds any table within the scraped page
my_table_row = my_table.find_all('tr') # Gets table rows from the table

<h3>Getting Headers of the Table</h3>

In [6]:
headings = []
for th in my_table_row[0].find_all('th'):
    headings.append(th.text.replace('\n',''))
headings

['Postal Code', 'Borough', 'Neighbourhood']

<h3>Extracting the table data</h3>

In [7]:
samples = []
for data in my_table_row:
    samples.append(data.text.split('\n'))
samples[:5]

[['', 'Postal Code', '', 'Borough', '', 'Neighbourhood', ''],
 ['', 'M1A', '', 'Not assigned', '', 'Not assigned', ''],
 ['', 'M2A', '', 'Not assigned', '', 'Not assigned', ''],
 ['', 'M3A', '', 'North York', '', 'Parkwoods', ''],
 ['', 'M4A', '', 'North York', '', 'Victoria Village', '']]

<h3>Loading the extracted and processed table into pandas DataFrame</h3>

In [32]:
df = pd.DataFrame(samples)
df.drop([0],inplace=True)
df.drop(columns = [0,2,4,6], inplace=True)
df.columns = headings
df.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h3>Removing the rows where Borough is not assigned</h3>

In [33]:
df_processed = df[df.Borough != 'Not assigned']
df_processed.reset_index(drop=True, inplace=True)
df_processed.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>Shape of the final DataFrame</h3>

In [14]:
df_processed.shape

(103, 3)

<h3>Loading the Geospatial Co-ordinates from csv to pandas DataFame</h3>

In [28]:
df_geo_coords = pd.read_csv("Geospatial_Coordinates.csv")
df_geo_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3>Merging both DataFrames using Outer Join</h3>

In [31]:
df_coords = pd.merge(df_processed, df_geo_coords, how='outer', on=['Postal Code'])
df_coords.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
